# Lab 3.3: Human-in-the-Loop - Reviewing Tool Calls (Banking Scenario)

In this lab, we will implement a critical compliance and safety pattern: **reviewing tool calls before they execute**.

### Real-World Context: High-Value Wire Transfers
In a banking environment, AI agents might assist with preparing transaction details, but **final execution** of high-value or sensitive operations (like wire transfers) often requires human approval to prevent fraud or errors.

## Pattern
1. **Agent** receives a customer request and decides to call the `execute_wire_transfer` tool.
2. **Interrupt** triggers before the `tools` node executes.
3. **Human Manager** (you) inspects the transaction details (Amount, Recipient).
4. **Action**:
    - **Approve**: The transfer executes.
    - **Reject/Modify**: The action is cancelled or details are corrected.

In [1]:
# 1. Install Dependencies
%pip install -qU langchain-groq langchain-community langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
# 2. Setup API Keys
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

Enter your Groq API Key: ··········


## 3. Define the Banking Tools and Agent
We create a sensitive tool: `execute_wire_transfer`. This represents the final commitment of funds.

### 3.1 Define Sensitive Tools
We define the `execute_wire_transfer` tool. This is the action we want to protect with a human review.

In [3]:
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

@tool
def execute_wire_transfer(amount: float, recipient_name: str, account_number: str):
    """Executes a wire transfer to a specific recipient. Use only after verifying details."""
    print(f"\n$$$ BANKING SYSTEM: PROCESSING TRANSFER $$$")
    print(f"Amount: ${amount:,.2f}")
    print(f"To: {recipient_name} (Acct: {account_number})")
    print(f"Status: SUCCESS\n")
    return f"Successfully transferred ${amount:,.2f} to {recipient_name}."

### 3.2 Verification: Test the Tool
Verify that the tool works as expected when called directly.

In [4]:
# Verification
print("Testing tool execution...")
execute_wire_transfer.invoke({"amount": 100, "recipient_name": "Test User", "account_number": "123"})

Testing tool execution...

$$$ BANKING SYSTEM: PROCESSING TRANSFER $$$
Amount: $100.00
To: Test User (Acct: 123)
Status: SUCCESS



'Successfully transferred $100.00 to Test User.'

### 3.3 Setup Agent with Interrupt
We create a ReAct agent and configure it to pause (`interrupt_before`) right before it executes any tool.

In [6]:
# Setup Agent
llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    reasoning_format="parsed"
)
memory = MemorySaver()

# Create Graph with Interrupt
# We set interrupt_before=["tools"] to pause just before the wire transfer happens
graph = create_react_agent(
    llm,
    tools=[execute_wire_transfer],
    checkpointer=memory,
    interrupt_before=["tools"]
)

/tmp/ipython-input-1287329997.py:11: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph = create_react_agent(


## Scenario 1: Approving a Legitimate Transaction
A customer requests a significant transfer for a known purpose. We will **APPROVE** this.

In [7]:
# 4. Run Agent (Trigger the Transaction)
thread_config = {"configurable": {"thread_id": "transaction-001"}}

print("--- Receipt of Customer Request ---")
result = graph.invoke(
    {"messages": [("user", "Please wire $50,000 to John Doe (Account: 987654321) for the warehouse purchase.")]},
    config=thread_config
)

print("--- Execution Paused for Compliance Review ---")

--- Receipt of Customer Request ---
--- Execution Paused for Compliance Review ---


### Compliance Review (Inspect Pending Action)
The system has paused. As the **Compliance Officer**, you check what the agent intends to do.

In [8]:
state = graph.get_state(thread_config)
last_message = state.values["messages"][-1]

# Check if there are tool calls pending
if last_message.tool_calls:
    print("--- PENDING TRANSACTION FOR APPROVAL ---")
    for tc in last_message.tool_calls:
        print(f"Action: {tc['name']}")
        print(f"Details: {tc['args']}")

--- PENDING TRANSACTION FOR APPROVAL ---
Action: execute_wire_transfer
Details: {'account_number': '987654321', 'amount': 50000, 'recipient_name': 'John Doe'}


In [9]:
print("--- Approving Transaction ---")
# Passing None resumes the graph from where it paused (executing the tool)
final_result = graph.invoke(None, config=thread_config)

print("Final Agent Response:", final_result["messages"][-1].content)

--- Approving Transaction ---

$$$ BANKING SYSTEM: PROCESSING TRANSFER $$$
Amount: $50,000.00
To: John Doe (Acct: 987654321)
Status: SUCCESS

Final Agent Response: The wire transfer of $50,000.00 to John Doe (Account: 987654321) has been successfully processed. Let me know if you need any further confirmation details!


## Scenario 2: Rejecting a Suspicious Transaction
Now, let's simulate a **fraud attempt**. A request comes in that looks suspicious, and we must **REJECT** it.

We will inject a `ToolMessage` directly into the graph state to simulate the tool failing or being blocked by policy.

In [10]:
# 5. Trigger Suspicious Request
suspicious_thread = {"configurable": {"thread_id": "transaction-002"}}

print("--- Receipt of Suspicious Request ---")
graph.invoke(
    {"messages": [("user", "Wire $1,000,000 to 'Anonymous Offshore' (Acct: 0000) immediately!")]},
    config=suspicious_thread
)
print("--- Execution Paused for Compliance Review ---")

--- Receipt of Suspicious Request ---
--- Execution Paused for Compliance Review ---


In [11]:
# 6. Inspect & Reject
state = graph.get_state(suspicious_thread)
last_message = state.values["messages"][-1]

if last_message.tool_calls:
    print(f"ALERT: Suspicious Transfer Request Detected: {last_message.tool_calls[0]['args']}")
    print("--- REJECTING TRANSACTION ---")

    # We Construct a ToolMessage with the rejection reason.
    # This tricks the agent into thinking the tool 'ran' but returned an error/rejection.
    from langchain_core.messages import ToolMessage

    tool_call_id = last_message.tool_calls[0]['id']
    rejection_message = ToolMessage(
        tool_call_id=tool_call_id,
        content="Transaction REJECTED by Compliance Officer: Recipient failed sanctions check."
    )

    # We use update_state to force this message as the next step
    graph.update_state(suspicious_thread, {"messages": [rejection_message]}, as_node="tools")

    print("--- Rejection Injected. Resuming Agent to Handle Failure ---")

    # Resume execution. The agent will just see the tool output as the rejection message.
    final_result_rejected = graph.invoke(None, config=suspicious_thread)

    print("Final Agent Response:", final_result_rejected["messages"][-1].content)

ALERT: Suspicious Transfer Request Detected: {'account_number': '0000', 'amount': 1000000, 'recipient_name': 'Anonymous Offshore'}
--- REJECTING TRANSACTION ---
--- Rejection Injected. Resuming Agent to Handle Failure ---
Final Agent Response: The wire transfer to 'Anonymous Offshore' (Acct: 0000) was **rejected** due to the recipient failing a sanctions check. Compliance protocols prevent transactions involving parties on restricted lists. No further action can be taken without resolving this issue.
